In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from ipynb.fs.full.utility_functions import gen_data_dict, get_data, get_cumsum, error_count, write_to_csv, opart, SquaredHingeLoss

np.random.seed(123)
torch.manual_seed(123)

In [2]:
# data
data = pd.read_csv('1.genome_learning_data/seq_features.csv')['count'].to_numpy()
data = np.log(np.log(data)).reshape(-1,1)
data = torch.FloatTensor(data)

target_df_1 = pd.read_csv('1.genome_learning_data/target_lambda_fold1.csv')
target_df_2 = pd.read_csv('1.genome_learning_data/target_lambda_fold2.csv')

targets_low_1  = target_df_1.iloc[:, 1:2].to_numpy()
targets_high_1 = target_df_1.iloc[:, 2:3].to_numpy()
targets_low_2  = target_df_2.iloc[:, 1:2].to_numpy()
targets_high_2 = target_df_2.iloc[:, 2:3].to_numpy()

targets_low_1  = torch.FloatTensor(targets_low_1)
targets_high_1 = torch.FloatTensor(targets_high_1)
targets_low_2  = torch.FloatTensor(targets_low_2)
targets_high_2 = torch.FloatTensor(targets_high_2)

y1 = torch.cat((targets_low_1, targets_high_1), dim=1)
y2 = torch.cat((targets_low_2, targets_high_2), dim=1)

In [3]:
# Define the linear model
class LinearModel(nn.Module):
    def __init__(self, input_size):
        super(LinearModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 1)

    def forward(self, x):
        return self.fc1(x)

In [4]:
# Instantiate the model, define custom loss function, and optimizer
model1 = LinearModel(input_size = 1)
model2 = LinearModel(input_size = 1)

squared_hinge_loss = SquaredHingeLoss()
optimizer1 = optim.Adam(model1.parameters(), lr=0.01, amsgrad=True)
optimizer2 = optim.Adam(model2.parameters(), lr=0.01, amsgrad=True)

# Training loop
for epoch in range(5000):
    # Forward pass
    outputs1 = model1(data)
    outputs2 = model2(data)
    
    # Compute the custom loss
    loss_1 = squared_hinge_loss(outputs1, y1)
    loss_2 = squared_hinge_loss(outputs2, y2)
    # loss_1_test = squared_hinge_loss(outputs1, targets_low_2, targets_high_2)
    # loss_2_test = squared_hinge_loss(outputs2, targets_low_1, targets_high_1)
    
    # Backward pass and optimization
    optimizer1.zero_grad()
    loss_1.backward()
    optimizer1.step()

    optimizer2.zero_grad()
    loss_2.backward()
    optimizer2.step()

    # # record
    # record_loss1_test.append(loss_1_test.item())
    # record_loss2_test.append(loss_2_test.item())

    # # save models
    # if loss_1_test < min_loss_1_test:
    #     min_loss_1_test = loss_1_test
    #     torch.save(model1.state_dict(), 'saved_models/model1_linear_best.pth')
    
    # if loss_2_test < min_loss_2_test:
    #     min_loss_2_test = loss_2_test
    #     torch.save(model2.state_dict(), 'saved_models/model2_linear_best.pth')
    
    # # Print the loss every 100 epochs
    # if (epoch) % 500 == 0:
    #     print(f'Epoch [{epoch:5d}/{epochs}], Loss_1: {loss_1.item():8.4f}, Loss_1_test: {loss_1_test.item():8.4f}, Loss_2: {loss_2.item():8.4f}, Loss_2_test: {loss_2_test.item():8.4f}')

In [5]:
torch.save(model1.state_dict(), '1.genome_saved_models/model1_linear_best.pth')
torch.save(model2.state_dict(), '1.genome_saved_models/model2_linear_best.pth')

In [6]:
# Load model1
model1 = LinearModel(input_size=1)
model1.load_state_dict(torch.load('1.genome_saved_models/model1_linear_best.pth'))
model1.eval()  # Set the model to evaluation mode

# Load model2
model2 = LinearModel(input_size=1)
model2.load_state_dict(torch.load('1.genome_saved_models/model2_linear_best.pth'))
model2.eval()  # Set the model to evaluation mode

LinearModel(
  (fc1): Linear(in_features=1, out_features=1, bias=True)
)

In [7]:
with torch.no_grad():
    ldas1 = model1(data).numpy().reshape(-1)
    ldas2 = model2(data).numpy().reshape(-1)

In [8]:
seqs   = gen_data_dict('sequence_label_data/genome/signals.gz')
labels = gen_data_dict('sequence_label_data/genome/labels.gz')

header = ['sequenceID', 'fold_1_total_labels', 'fold_2_total_labels', 'fold_1_errs', 'fold_2_errs']

for i in range(len(seqs)):
    # generate data
    sequence, neg_start_1, neg_end_1, pos_start_1, pos_end_1, neg_start_2, neg_end_2, pos_start_2, pos_end_2 = get_data(i, seqs=seqs, labels=labels)
    sequence_length = len(sequence)-1

    # vectors of cumulative sums
    y, z = get_cumsum(sequence)

    # get total labels
    fold1_total_labels = len(neg_start_1) + len(pos_start_1)
    fold2_total_labels = len(neg_start_2) + len(pos_start_2)

    # run each lambda and record it into csv file
    row  = [i, fold1_total_labels, fold2_total_labels]

    chpnt_fold1 = opart(10**ldas2[i], sequence)
    chpnt_fold2 = opart(10**ldas1[i], sequence)

    err_1 = error_count(chpnt_fold1, neg_start_1, neg_end_1, pos_start_1, pos_end_1)
    err_2 = error_count(chpnt_fold2, neg_start_2, neg_end_2, pos_start_2, pos_end_2)
    
    row.append(sum(err_1))
    row.append(sum(err_2))

    write_to_csv('1.genome_learning_output/linear.csv', header, row)